### Μέρος 2: Εντοπισμός Χωρο-χρονικών Σημείων Ενδιαφέροντος και Εξαγωγή Χαρακτηριστικών σε Βίντεο Ανθρωπίνων Δράσεων

Ανδρέας Γεωργής 03115194 

Κούβαρης Σταύρος 03114090

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import math
import cv20_lab1_part2_utils  as ut
from scipy.ndimage import convolve1d
import cv20_lab2_part2_material as l2
from cv20_lab2_part2_material import cv20_lab2_2_utils  as ut2
from sklearn.preprocessing import normalize
import LucasKanade as lk

Εισαγωγή βίντεο

In [9]:
path = "C:\\Users\\Andrew\\anaconda3\\envs\\cv_lab\\CV20_Lab2\\cv20_lab2_part2_material\\boxing\\person06_boxing_d1_uncomp.avi"
nframes = 200
video = ut2.read_video(path,nframes,0),
vid = video[0]

#### 2.1 Χωρο-χρονικά Σημεία Ενδιαφέροντος


###### Υλοποιήση 3D Harris-Stephens και Gabor ανιχνευτή

In [4]:
def Harris_Stephens(video ,sigma, s , t , k ,theta):
    n = int((np.ceil(3*sigma) * 2 + 1))
    # create 2D Gaussian spatial kernel and 1D Gaussian time kernel
    G_1D = cv2.getGaussianKernel(n, sigma)
    G_2D = G_1D @ G_1D.T
    nt =  int((np.ceil(3*t) * 2 + 1))
    G_t = cv2.getGaussianKernel(nt, t)
    
    # weights = [-1,0,1]T mask for calculating derivatives 
    weights = np.array([-1,0,1])
    L = np.empty(video.shape)
    
    # calculate L
    for i in range(video.shape[2]):
        L[:,:,i] = cv2.filter2D(cv2.filter2D(video[:,:,i],-1,G_2D),-1,G_t)
       
    # calculate second order derivatives of L 
    Lx = convolve1d(L,weights.T,axis=0)
    Ly = convolve1d(L,weights.T,axis=1)
    Lt = convolve1d(L,weights.T,axis=2)
    Lxx = Lx**2
    Lxy = Lx*Ly
    Lxt = Lx*Lt
    Lyy= Ly**2
    Lyt = Ly*Lt
    Ltt = Lt**2
    
    # calculate M
    Mxx = cv2.filter2D(cv2.filter2D(Lxx,-1,G_2D),-1,G_t)
    Myy = cv2.filter2D(cv2.filter2D(Lyy,-1,G_2D),-1,G_t)
    Mtt= cv2.filter2D(cv2.filter2D(Ltt,-1,G_2D),-1,G_t)
    Mxt = cv2.filter2D(cv2.filter2D(Lxt,-1,G_2D),-1,G_t)
    Myt = cv2.filter2D(cv2.filter2D(Lyt,-1,G_2D),-1,G_t)
    Mxy = cv2.filter2D(cv2.filter2D(Lxy,-1,G_2D),-1,G_t)
    # Corner criterion
    trM = Mxx + Myy + Mtt
    dtM = Mxx * Myy *Mtt + 2*Mxy*Mxt*Myt - Mxx*(Myt**2) - Myy*(Mxt**2) - Mtt*(Mxy**2)
    
    H = np.abs(dtM - k*trM**3)
    ns = np.ceil(3*sigma)*2 + 1
    B_sq = ut.disk_strel(ns)
    indeces = []
    edges = np.empty(H.shape)
    Cond1 = (H[:,:,0]==cv2.dilate(H[:,:,0],B_sq))
    
    # Condition 2
    Cond2 = (H[:,:,0]> theta*H[:,:,0].max())
    
    # Find edges and return the corresponding points. 
    edges[:,:,0]= np.logical_and(Cond1, Cond2)
    indeces.append(np.where(edges[:,:,0] == 1))
    sigm = np.full(indeces[0][0].shape,sigma)
    T = np.full(indeces[0][0].shape,0)
    bla = np.vstack((indeces[0][0],indeces[0][1],T,sigm))
    points = bla.T
    for i in range(1,H.shape[2]):
        Cond1 = (H[:,:,i]==cv2.dilate(H[:,:,i],B_sq))
        # Condition 2
        Cond2 = (H[:,:,i] > theta*H[:,:,i].max())
        # Edges
        edges[:,:,i] = np.logical_and(Cond1, Cond2)
        indeces.append(np.where(edges[:,:,i] == 1))
        sigm = np.full(indeces[i][0].shape,sigma)
        T = np.full(indeces[i][0].shape,i)
        bla = np.vstack((indeces[i][1],indeces[i][0],T,sigm))
        points = np.concatenate((points,bla.T))
    return points    

In [5]:
def Gabor3D(video ,sigma, t,theta):
    w = 4/t
    start = int(np.ceil(-2*t))
    stop = int(np.ceil(2*t))
    time = np.asarray([i for i in range(start,stop)])
    hev = np.cos(2*np.pi*time*w) * np.exp(-(time**2) / (2*(t**2)))
    hod = np.sin(2*np.pi*time*w) * np.exp(-(time**2) / (2*(t**2)))
    H = np.empty(video.shape,dtype='uint8')
    
    # reshape for be 2D array for normalize
    hev = np.reshape(hev,(len(hev),1))
    hod = np.reshape(hod,(len(hod),1))
    
    # normalize with l1 norm
    hev_l1 =normalize(hev,norm='l1')
    hod_l1 =normalize(hod,norm='l1')
    hev = hev / hev_l1
    hod = hod / hod_l1
    
    # filtering frames with 2D Gaussian filters
    n = int((np.ceil(3*sigma) * 2 + 1))
    G_1D = cv2.getGaussianKernel(n, sigma)
    G_2D = G_1D @ G_1D.T
    for i in range(video.shape[2]):
        Iev = cv2.filter2D(cv2.filter2D(video[:,:,i],-1,G_2D),-1,hev)
        #Iev = cv2.filter2D(Iev,-1,hev)
        Iod = cv2.filter2D(cv2.filter2D(video[:,:,i],-1,G_2D),-1,hod)
        H[:,:,i] = Iev**2 + Iod**2
    
    ns = np.ceil(3*sigma)*2 + 1
    B_sq = ut.disk_strel(ns)
    indeces = []
    edges = np.empty(H.shape,dtype='uint8')
    Cond1 = (H[:,:,0]==cv2.dilate(H[:,:,0],B_sq))
    
    # Condition 2
    Cond2 = (H[:,:,0]> theta*H[:,:,0].max())
    
    # Find edges and return the corresponding points. 
    edges[:,:,0]= np.logical_and(Cond1, Cond2)
    indeces.append(np.where(edges[:,:,0] == 1))
    sigm = np.full(indeces[0][0].shape,sigma)
    T = np.full(indeces[0][0].shape,0)
    bla = np.vstack((indeces[0][0],indeces[0][1],T,sigm))
    points = bla.T
    for i in range(1,H.shape[2]):
        Cond1 = (H[:,:,i]==cv2.dilate(H[:,:,i],B_sq))
        # Condition 2
        Cond2 = (H[:,:,i] > theta*H[:,:,i].max())
        # Edges
        edges[:,:,i] = np.logical_and(Cond1, Cond2)
        indeces.append(np.where(edges[:,:,i] == 1))
        sigm = np.full(indeces[i][0].shape,sigma)
        T = np.full(indeces[i][0].shape,i)
        bla = np.vstack((indeces[i][1],indeces[i][0],T,sigm))
        points = np.concatenate((points,bla.T))
        
    return points


#### Ανίχνευση τοπικών χαρακτηριστικών με χρήση του Harris-Stephens ανιχνευτή.

In [11]:
sigma = 3
s = 1.5
t = 0.7
k = 0.0005
theta = 0.05

points = Harris_Stephens(vid,sigma , s , t, k ,theta)
ut2.show_detection(vid,points,save_path="C:\\Users\\Andrew\\anaconda3\\envs\\cv_lab\\CV20_Lab2\\Results\\Boxing2")

#### Ανίχνευση σημείων ενδιαφέροντος χρησιμοποιώντας Gabor φίλτρα.

In [12]:
sigma = 2.5
t = 1.3

theta = 0.2
points = Gabor3D(vid,sigma, t,theta)
path = "C:\\Users\\Andrew\\anaconda3\\envs\\cv_lab\\CV20_Lab2\\Results\\Boxing_Gabor2"
ut2.show_detection(vid,points,save_path=path)

C:\Users\Andrew\anaconda3\envs\cv_lab\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


#### 2.2 Χωρο-χρονικοί Ιστογραφικοί Περιγραφητές
2.2.1 - 2.2.3

#### 2.2 Χωρο-χρονικοί Ιστογραφικοί Περιγραφητές

In [13]:
def Descriptor(video,isHOG,rho,nbins, epsilon, d_x0, d_y0,neighbor_size,points,desc_cells_per_dim=2,reps=15):
    weights = np.array([-1,0,1])
    # Normalize to [0,1]
    Gx, Gy = np.empty(video.shape),np.empty(video.shape)
    Im = video.astype(np.float)/255
    dx,dy = np.empty(Im.shape),np.empty(Im.shape)
    # Calculate the derivatives of each frame using the mask [-1,0,1]T
    if isHOG:
        Ix = convolve1d(Im,weights,axis=0)
        Iy = convolve1d(Im,weights,axis=1)
    else:
        for i in range(video.shape[2] - 1 ):
            dx[:,:,i],dy[:,:,i] = lk.Lucas_Kanade(Im[:,:,i],Im[:,:,i+1], d_x0, d_y0, rho, epsilon, reps, False, i+1, "saveFolder")
    
    window = int(np.floor(np.ceil(neighbor_size)))
    descriptors = [] 
    # Define the boundaries of Gx and Gy 
    for i in range(1,points.shape[0]):
        px = int(points[i,0])
        py = int(points[i,1])
        t = int(points[i,2])
        if t == Im.shape[2] and isHOG == 0 :
            continue
        if px - window < 0:
            lower_x_bound = 0
        else:
            lower_x_bound =  int(px - window)
    
        if px + window > Im.shape[1] :
            upper_x_bound = Im.shape[1]
        else:
            upper_x_bound =  int(px + window)
    
        if py - window < 0:
            lower_y_bound = 0
        elif py > Im.shape[0]:
            continue
        else:
            lower_y_bound =  int(py - window)
            
        if py + window > Im.shape[0]:
            upper_y_bound = Im.shape[0]
        else:
            upper_y_bound =  int(py + window)
        
        if isHOG:
            Gx = Ix[lower_y_bound: upper_y_bound, lower_x_bound: upper_x_bound, t]
            Gy = Iy[lower_y_bound: upper_y_bound, lower_x_bound: upper_x_bound, t]
        else:
            Gx = dx[lower_y_bound: upper_y_bound, lower_x_bound: upper_x_bound, t]
            Gy = dy[lower_y_bound: upper_y_bound, lower_x_bound: upper_x_bound, t]

        desc_dim = np.array([desc_cells_per_dim, desc_cells_per_dim] )
        
        descriptors.append(ut2.orientation_histogram(Gx, Gy, nbins, desc_dim,0,0))
        
    return descriptors


Δημιουργία συνάρτησης feature_extracrion η οποία θα δέχεται ένα βίντεο, μια παράμετρο isHarris που επιλέγει ποιος ανιχνευτής θα χρησιμοποιηθεί για την εύρεση των σημείων ενδιαφέροντος (1 για τον Harris, 0 για τον Gabor), καθώς και τις απαραίτητες παραμέτρους που χρειάζονται οι ανιχνευτές.
Η έξοδος της συνάρτησης είναι ο HOG/HOF περιγραφητής σε μορφή 2D numpy array.

In [14]:
def feature_extraction(video,isHarris,sigma,nbins, s , t, k ,theta,rho, epsilon, d_x0, d_y0,neighbor_size,desc_cells_per_dim, reps ):
    # calculate the points
    if isHarris:
        points = Harris_Stephens(video,sigma , s , t, k ,theta)
    else:
        points = Gabor3D(video,sigma, t,theta)
   
    # apply HOG and HOF descriptors
    HoG_desc = Descriptor(video,1,rho,nbins, epsilon, d_x0, d_y0,neighbor_size,points,desc_cells_per_dim ,reps)
    HoF_desc = Descriptor(video,0,rho,nbins, epsilon, d_x0, d_y0,neighbor_size,points,desc_cells_per_dim ,reps)
    
    # convert to 2D numpy array
    HoG_Desc = np.asarray(HoG_desc,dtype='uint8')
    HoF_Desc = np.asarray(HoF_desc,dtype='uint8')
    
    # create HOG/HOF descriptor
    HoG_HoF = np.empty((2*HoG_Desc.shape[0],HoG_Desc.shape[1]),dtype='uint8')
    HoG_HoF = np.concatenate((HoG_Desc,HoF_Desc))
    
    return HoG_HoF


Η συνάρτηση create_train_test παίρνει ως όρισμα το path του αρχείου στο οποίο βρίσκονται τα αρχεία για το training και test σετ και επιστρέφει σε μορφή τα αρχεία των βίντεο καθώς και τα labels του κάθε βίντεο.

In [15]:
def create_train_test(path):
    training_video_paths = []
    test_video_paths = []
    training_labels = []
    test_labels = []
    with open(path + "\\traininng_videos.txt", "r") as filestream:
        for line in filestream:
            currentline = line.split("_")
            paths = path + currentline[1] + "\\" + line
            training_video_paths.append(paths)
            training_labels.append(currentline[1])

    with  open(path + "\\test_videos.txt", "r") as filestream:
        for line in filestream:
            currentline = line.split("_")
            paths = path + currentline[1] + "\\" + line
            test_video_paths.append(paths) 
            test_labels.append(currentline[1])
    
    return training_video_paths, training_labels,test_video_paths,test_labels

#### 2.3: Κατασκευή Bag of Visual Words και χρήση Support Vector Machines για την ταξινόμηση δράσεων
##### Ταξινόμηση δράσεων χρησιμοποιώντας  τον Harris-Stephens ανιχνευτή

In [91]:
nframes = 200
nbins = 9
isHarris = 1
sigma = 4
neighbor_size = 4*sigma
d_x0, d_y0 = 0,0
rho, epsilon = 1, 0.05
s = 1.5
t = 0.7
k = 0.005
theta = 0.05
D = 500 # initialize the number of centroids that will be used for the BoVW 
desc_cells_per_dim = 2
reps = 15
####################################################3
##### Create train and test datasets
##### The training_video_paths is a list that holds the paths for reading the video
##### Respectively the test_video_paths
path = "C:\\Users\\Andrew\\anaconda3\\envs\\cv_lab\\CV20_Lab2\\cv20_lab2_part2_material\\"
training_video_paths,train_labels,test_video_paths,test_labels = create_train_test(path)

for path in training_video_paths:
    video = ut2.read_video(path,nframes,0)
    desc_train = feature_extraction(video,isHarris,sigma,nbins, s , t, k ,theta,rho, epsilon, d_x0, d_y0,neighbor_size,desc_cells_per_dim, reps)
    
for path in test_video_paths:
    print(path)
    video = ut2.read_video(path,nframes,0)
    desc_test = feature_extraction(video,isHarris,sigma,nbins, s , t, k ,theta,rho, epsilon, d_x0, d_y0,neighbor_size,desc_cells_per_dim, reps)

############################################################3
####### Calculate the BoVW for training and test datasets 
####### Find the accuracy and predictions using svm classifier
bow_train, bow_test = ut2.bag_of_words(desc_train, desc_test, num_centers=D)
    
accuracy, pred = ut2.svm_train_test(bow_train, train_labels, bow_test, test_labels)
    

Display the results for using Harris-Stephens detector and HOG/HOF descriptor

In [102]:
print('The accuracy is:',100.0*accuracy,"%")
print('predictions:',pred)

The accuracy is: 69.23076923076923 %
predictions: ['running' 'walking' 'boxing' 'running' 'running' 'boxing' 'boxing'
 'boxing' 'boxing' 'boxing' 'walking' 'walking' 'boxing']


#### Ταξινόμηση δράσεων χρησιμοποιώντας τον Gabor ανιχνευτή

In [106]:
nframes = 200
nbins = 9
isHarris = 0
sigma = 2.5
neighbor_size = 4*sigma
d_x0, d_y0 = 0,0
rho, epsilon = 1, 0.05
s = 1.5
t = 1.3
k = 0.005
theta = 0.2
D = 500 
desc_cells_per_dim = 2
reps = 15
path = "C:\\Users\\Andrew\\anaconda3\\envs\\cv_lab\\CV20_Lab2\\cv20_lab2_part2_material\\"
training_video_paths,train_labels,test_video_paths,test_labels = create_train_test(path)
desc_train_Gabor = []
for path in training_video_paths[3:]:
    video = ut2.read_video(path,nframes,0)
    feats = feature_extraction(video,isHarris,sigma,nbins, s , t, k ,theta,rho, epsilon, d_x0, d_y0,neighbor_size,desc_cells_per_dim, reps)
    desc_train_Gabor.append(feats)

#desc_test_Gabor = []    
for path in test_video_paths:
    video = ut2.read_video(path,nframes,0)
    feats = feature_extraction(video,isHarris,sigma,nbins, s , t, k ,theta,rho, epsilon, d_x0, d_y0,neighbor_size,desc_cells_per_dim, reps)
    desc_test_Gabor.append(feats)

bow_train_Gabor, bow_test_Gabor = ut2.bag_of_words(desc_train_Gabor, desc_test_Gabor, num_centers=D)
    
accuracy, pred = ut2.svm_train_test(bow_train_Gabor, train_labels, bow_test_Gabor, test_labels)
    

In [108]:
print('The accuracy is:',100.0*accuracy,"%") 
print('predictions:', pred )

The accuracy is: 76.92307692307692 %
predictions: ['running' 'running' 'running' 'running' 'running' 'running' 'boxing' 'boxing' 'boxing' 'boxing' 'walking' 'walking' 'running']
